In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import matplotlib.pyplot as plt

In [3]:
torch.manual_seed(42)  # for reproducibility

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zalando-research/fashionmnist")

print("Path to dataset files:", path)

Path to dataset files: /home/darshan39/.cache/kagglehub/datasets/zalando-research/fashionmnist/versions/4


In [5]:
df = pd.read_csv(f"{path}/fashion-mnist_train.csv")

In [6]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
x = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [9]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,
    random_state=42,
)

In [10]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [11]:
x_train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00392157, 0.        ,
        0.        ]], shape=(48000, 784), dtype=float32)

In [12]:
class FashionMNISTDataset(Dataset):

    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [13]:
train_dataset = FashionMNISTDataset(x_train, y_train)

In [14]:
train_dataset[0]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2275,
         0.5333, 0.0000, 0.0

In [16]:
test_dataset = FashionMNISTDataset(x_test, y_test)

In [17]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [18]:
class myNN(nn.Module):
    def __init__(self, num_features):
        super(myNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
        
    def forward(self, x):
        return self.model(x)

In [19]:
learning_rate = 0.1
num_epochs = 100

In [20]:
model = myNN(x_train.shape[1])

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [21]:
for epoch in range(num_epochs):

    total_epoch_loss = 0

    for batch_features, batch_labels in train_loader:

        # Forward pass
        outputs = model(batch_features)


        loss = criterion(outputs, batch_labels)

        # Backward pass and optimization
        optimizer.zero_grad()


        loss.backward()


        optimizer.step()

        total_epoch_loss += loss.item()
    
    avg_epoch_loss = total_epoch_loss / len(train_loader)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_epoch_loss:.4f}")
    

Epoch [1/100], Loss: 0.6352
Epoch [2/100], Loss: 0.4299
Epoch [3/100], Loss: 0.3865
Epoch [4/100], Loss: 0.3592
Epoch [5/100], Loss: 0.3374
Epoch [6/100], Loss: 0.3227
Epoch [7/100], Loss: 0.3083
Epoch [8/100], Loss: 0.2960
Epoch [9/100], Loss: 0.2853
Epoch [10/100], Loss: 0.2746
Epoch [11/100], Loss: 0.2682
Epoch [12/100], Loss: 0.2580
Epoch [13/100], Loss: 0.2508
Epoch [14/100], Loss: 0.2438
Epoch [15/100], Loss: 0.2390
Epoch [16/100], Loss: 0.2319
Epoch [17/100], Loss: 0.2271
Epoch [18/100], Loss: 0.2209
Epoch [19/100], Loss: 0.2151
Epoch [20/100], Loss: 0.2087
Epoch [21/100], Loss: 0.2053
Epoch [22/100], Loss: 0.2017
Epoch [23/100], Loss: 0.1952
Epoch [24/100], Loss: 0.1933
Epoch [25/100], Loss: 0.1879
Epoch [26/100], Loss: 0.1841
Epoch [27/100], Loss: 0.1809
Epoch [28/100], Loss: 0.1782
Epoch [29/100], Loss: 0.1718
Epoch [30/100], Loss: 0.1695
Epoch [31/100], Loss: 0.1685
Epoch [32/100], Loss: 0.1669
Epoch [33/100], Loss: 0.1604
Epoch [34/100], Loss: 0.1599
Epoch [35/100], Loss: 0

In [22]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_features, batch_labels in test_loader:
        outputs = model(batch_features)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 88.53%
